https://github.com/huggingface/transformers/blob/0ae96ff8a7e2d371242452d81bee85da8df202f5/examples/text-generation/run_generation.py

# get a bunch of user comments

In [1]:
# check praw.ini in root
import praw

In [2]:
reddit = praw.Reddit()

In [3]:
user_name = 'suncoasthost'

In [4]:
user = reddit.redditor(user_name)

In [5]:
def get_context(comment, reddit):
    submission = reddit.submission(id=comment.link_id)
    parent_comment = None
    if not comment.parent_id == comment.link_id:
        # not a top level comment, try to retrieve parent comment
        parent_comment = reddit.comment(id=comment.parent_id.replace('t1_', ''))
    return parent_comment, submission

In [6]:
# args for user.comments.new()
# https://praw.readthedocs.io/en/latest/code_overview/other/listinggenerator.html#praw.models.ListingGenerator
# limit – default 100, max 1000
comment_context = [
    (comment, get_context(comment, reddit)) 
    for comment in user.comments.new()
]

In [7]:
comment_context[0]

(Comment(id='fw9xf8b'), (Comment(id='fw9win2'), Submission(id='t3_hgtxpz')))